## Gain and miss. noise params

author: steeve.laquitaine@epfl.ch

In [5]:
# import libs
%load_ext autoreload
%autoreload 2

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore")

import os
import numpy as np
from matplotlib import pyplot as plt
import spikeinterface as si
import pandas as pd
import seaborn as sns

# move to PROJECT PATH
PROJ_PATH = "/gpfs/bbp.cscs.ch/project/proj85/home/laquitai/preprint_2023/"
os.chdir(PROJ_PATH)

from src.nodes import utils
from src.nodes.utils import get_config
from src.nodes.validation import snr
from src.nodes.validation import amplitude as amp

# neuropixels (biophysical model)
cfg_ns, param_conf_ns = get_config("silico_neuropixels", "concatenated").values()
RAW_PATH_ns = cfg_ns["probe_wiring"]["40m"]["output_noise_fitd_gain_fitd_adj10perc_less_int16"]
PREP_PATH_ns = cfg_ns["preprocessing"]["output"]["full"]["trace_file_path_gain_fitd_adj10perc_less_noise_fitd_int16"]
FIT_PMS_PATH_ns = cfg_ns["preprocessing"]["fitting"]["missing_noise_path"]

# neuropixels (evoked biophysical model)
cfg_ne, _ = get_config("silico_neuropixels", "stimulus").values()
RAW_PATH_ne = cfg_ne["probe_wiring"]["output"]
PREP_PATH_ne = cfg_ne["preprocessing"]["output"]["full"]["trace_file_path_gain_fitd_adj10perc_less_noise_fitd_int16"]
FIT_PMS_PATH_ne = cfg_ne["preprocessing"]["fitting"]["missing_noise_path"]

# neuropixels (synthetic, Buccino)
cfg_nb, param_conf_nb = get_config("buccino_2020", "2020").values()
RAW_PATH_nb = cfg_nb["probe_wiring"]["10m"]["output_gain_fitd_int16"]
PREP_PATH_nb = cfg_nb["preprocessing"]["output"]["trace_file_path_gain_ftd"]
FIT_PMS_PATH_nb = cfg_nb["preprocessing"]["fitting"]["tuned_gain"]

# axes
FIG_SIZE = (1.5, 1.6)
COLOR_VIVO = np.array([153, 153, 153]) / 255
COLOR_SILI = np.array([228, 26, 28]) / 255
COLOR_BUCCI = np.array([55, 126, 184]) / 255
COLOR_NE = [1, 0.49, 0]
COLOR_MEDIAN = (1, 0.76, 0)
plt.rcParams["font.family"] = "Arial"
plt.rcParams["font.size"] = 6  # 5-7 with Nature neuroscience as reference
plt.rcParams["lines.linewidth"] = 0.5
plt.rcParams["axes.linewidth"] = 0.5 #1
plt.rcParams["axes.spines.top"] = False
plt.rcParams["xtick.major.width"] = 0.5 #0.8 #* 1.3
plt.rcParams["xtick.minor.width"] = 0.5 #0.8 #* 1.3
plt.rcParams["ytick.major.width"] = 0.5 #0.8 #* 1.3
plt.rcParams["ytick.minor.width"] = 0.5 #0.8 #* 1.3
plt.rcParams["xtick.major.size"] = 3.5 * 1.1
plt.rcParams["xtick.minor.size"] = 2 * 1.1
plt.rcParams["ytick.major.size"] = 3.5 * 1.1
plt.rcParams["ytick.minor.size"] = 2 * 1.1
# legend
savefig_cfg = {"transparent":True, "dpi":300}
legend_cfg = {"frameon": False, "handletextpad": 0.1}
tight_layout_cfg = {"pad": 0.5}
LG_FRAMEON = False              # no legend frame
savefig_cfg = {"transparent":True, "dpi": 300}

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
2024-07-12 10:32:56,118 - root - utils.py - get_config - INFO - Reading experiment config.


2024-07-12 10:32:56,207 - root - utils.py - get_config - INFO - Reading experiment config. - done
2024-07-12 10:32:56,208 - root - utils.py - get_config - INFO - Reading experiment config.
2024-07-12 10:32:56,240 - root - utils.py - get_config - INFO - Reading experiment config. - done
2024-07-12 10:32:56,242 - root - utils.py - get_config - INFO - Reading experiment config.
2024-07-12 10:32:56,266 - root - utils.py - get_config - INFO - Reading experiment config. - done


## Load dataset

In [62]:
def get_fit_prms(fit_path, layer):
    gain = np.load(fit_path + layer + ".npy", allow_pickle=True).item()["gain"]
    tuning = np.load(fit_path + layer + ".npy", allow_pickle=True).item()[
        "gain_adjustm"
    ]
    gain *= tuning
    missing_noise = np.load(fit_path + layer + ".npy", allow_pickle=True).item()[
        "missing_noise_rms"
    ]
    # report
    print(
        f"- {layer} gain: {np.round(gain)}, missing noise rms: {np.round(missing_noise,2)} uV"
    )

## Report

In [77]:
# spontaneous model
print("NPX-Biophy-spontaneous" + ":")
get_fit_prms(
    FIT_PMS_PATH_ns,
    "L1",
)
get_fit_prms(FIT_PMS_PATH_ns, "L2_3")
get_fit_prms(FIT_PMS_PATH_ns, "L4")
get_fit_prms(FIT_PMS_PATH_ns, "L5")
get_fit_prms(FIT_PMS_PATH_ns, "L6")

# Unit-test: they should be identical to the
# spontaneous model's (same probe with only stimulus
# added)
print("\nNPX-Biophy-evoked" + ":")
get_fit_prms(
    FIT_PMS_PATH_ne,
    "L1",
)
get_fit_prms(FIT_PMS_PATH_ne, "L2_3")
get_fit_prms(FIT_PMS_PATH_ne, "L4")
get_fit_prms(FIT_PMS_PATH_ne, "L5")
get_fit_prms(FIT_PMS_PATH_ne, "L6")

# Synthetic
# missing noise is already modeled
print("\nNPX-Synth-spontaneous" + ":")
print(f"L5 gain: {np.round(np.load(FIT_PMS_PATH_nb, allow_pickle=True).item(),2)}")

NPX-Biophy-spontaneous:
- L1 gain: 677.0, missing noise rms: 2.84 uV
- L2_3 gain: 677.0, missing noise rms: 3.41 uV
- L4 gain: 677.0, missing noise rms: 3.71 uV
- L5 gain: 677.0, missing noise rms: 3.56 uV
- L6 gain: 677.0, missing noise rms: 3.81 uV

NPX-Biophy-evoked:
- L1 gain: 677.0, missing noise rms: 2.84 uV
- L2_3 gain: 677.0, missing noise rms: 3.41 uV
- L4 gain: 677.0, missing noise rms: 3.71 uV
- L5 gain: 677.0, missing noise rms: 3.56 uV
- L6 gain: 677.0, missing noise rms: 3.81 uV

NPX-Synth-spontaneous:
L5 gain: 0.41
